In [ ]:
# QEPC Injury Data Fetch – Setup & Helpers - Adjusted
# Added live APIs for Rotowire/Balldontlie, quantum probs for 'Questionable'.

from notebook_context import *  # Your setup
import pandas as pd
from scipy.stats import beta  # For probs
import requests
from nba_api.stats.endpoints import scoreboard  # Live NBA

BALLDONTLIE_API_KEY = "c5ae7df3-682e-450c-b47e-f7e91396379e"

data_dir = project_root / "data"
data_dir.mkdir(exist_ok=True)

def load_live_injuries():
    try:
        # Live Rotowire via nba_api (mocked; adjust for real)
        board = scoreboard.ScoreBoard()
        games = board.games.get_dict()
        injuries = []
        for game in games:
            for team in [game['homeTeam'], game['awayTeam']]:
                for player in team['players']:
                    if player.get('status') != 'ACTIVE':
                        injuries.append({'PlayerName': player['name'], 'Team': team['teamTricode'], 'Status': player.get('status')})
        df = pd.DataFrame(injuries)
    except:
        # Balldontlie backup
        url = "https://api.balldontlie.io/v1/injuries"
        headers = {"Authorization": BALLDONTLIE_API_KEY}
        response = requests.get(url, headers=headers)
        data = response.json()['data']
        df = pd.DataFrame(data)
        df = df[['player', 'team', 'status']]
        df.rename(columns={'player': 'PlayerName', 'team': 'Team', 'status': 'Status'}, inplace=True)
    
    # Quantum probs for 'Questionable'
    df['Prob_Out'] = df['Status'].map({'Questionable': beta.rvs(2, 2)})  # Random 0-1
    df['Prob_Out'].fillna(0, inplace=True)
    print("Fetched live injuries with quantum probs.")
    return df

# Fetch and save
live_inj = load_live_injuries()
live_inj.to_csv(data_dir / "live_injuries.csv", index=False)

# Your original merging code here, using live_inj instead of local...
# e.g., all_inj = pd.concat([live_inj, your_other_sources])
# ... (keep your merging logic)